In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/raw/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [5]:
df_yellow = spark.read.parquet('data/raw/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [6]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [7]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')
# df_join.show()

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [12]:
df_join = spark.read.parquet('data/report/revenue/total')

In [13]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [14]:
df_zones = spark.read.parquet('zones/')

In [15]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [17]:
df_result.drop('LocationID', 'zone').write.parquet('data/report/revenue/revenue-zones')

In [19]:
df_result.head(10)

[Row(hour=datetime.datetime(2020, 1, 1, 0, 0), zone=263, green_amount=None, green_number_records=None, yellow_amount=14.76, yellow_number_records=1, LocationID='263', Borough='Manhattan', Zone='Yorkville West', service_zone='Yellow Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 2, 0), zone=43, green_amount=None, green_number_records=None, yellow_amount=17.8, yellow_number_records=1, LocationID='43', Borough='Manhattan', Zone='Central Park', service_zone='Yellow Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 2, 0), zone=68, green_amount=None, green_number_records=None, yellow_amount=48.16, yellow_number_records=3, LocationID='68', Borough='Manhattan', Zone='East Chelsea', service_zone='Yellow Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 2, 0), zone=75, green_amount=18.3, green_number_records=1, yellow_amount=9.3, yellow_number_records=1, LocationID='75', Borough='Manhattan', Zone='East Harlem South', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2020, 1, 1, 2, 0), zone=82,